In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Paths to data

In [ ]:
train_dir='data/train/'
test_dir='data/test/'

# ImageDataGenerator for data augmentation

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen=ImageDataGenerator(rescale=1./255)

# Load images from directory

In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

# Model architecture

In [ ]:
model=models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(train_generator.num_classes,activation='softmax')
])

# Compile,train & save the model

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_generator,validation_data=test_generator,epochs=15)
model.save('models/sign_language_model.h5')

# Evaluate the model

In [ ]:
test_loss,test_acc=model.evaluate(test_generator)
print(f'Test accuracy:{test_acc:.2f}')

# Confusion matrix

In [ ]:
y_true=test_generator.classes
y_pred=np.argmax(model.predict(test_generator),axis=-1)
conf_matrix=confusion_matrix(y_true,y_pred)
class_labels=list(test_generator.class_indices.keys())
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrix,annot=True,fmt='d',cmap='Blues',xticklabels=class_labels,yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification report

In [ ]:
print(classification_report(y_true,y_pred,target_names=class_labels))